In [ ]:
# this is the collection_data.py

import numpy as np
from sklearn.model_selection import train_test_split
import os
import pandas as pd


# This class is going to prepare the data into a suitable format
# Credit due to : https://www.kaggle.com/datasets/grassknoted/asl-alphabet
def dataset_preparation(
        # this is my local path
        dataset_path=r'E:\Documents\King Of The Software Engineers\ML\DATA 5100 Programming for Data Science\Final '
                     r'Porject'):
    dataset_info = {
        'number_of_images': 0,
        'classes': [],
        'train_split': 0.8,
        'validation_split': 0.2
    }

    # Iterate dataset directory
    for root, dirs, files in os.walk(dataset_path):

        # Counts the number of image files in a directory with the given extensions

        dataset_info['number_of_images'] += len([f for f in files if f.endswith(('.jpg', '.png', '.jpeg'))])
        # # Capture only top directories
        if not dataset_info['classes'] and dirs:
            dataset_info['classes'] = dirs

    return dataset_info


def main():
    # my local path
    DATASET_PATH = (r'E:\Documents\King Of The Software Engineers\ML\DATA 5100 Programming for Data Science\Final '
                    r'Porject\archive\asl_alphabet_train\asl_alphabet_train')
    # Gather and preprocess data
    dataset_information = dataset_preparation(DATASET_PATH)
    print("Data Collection and Preprocessing Summary:")
    print(f"Number of Imges: {dataset_information['number_of_images']}")
    print(f"Available classes: {dataset_information['classes']}")
    print(f" The split: {dataset_information['train_split'] * 100}%")
    print(f"The validation split: {dataset_information['validation_split'] * 100}%")


if __name__ == '__main__':
    main()


In [ ]:
# This is the model_training.py

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


class SignModel:
    def __init__(self, input_shape=(64, 64, 3),
                 num_classes=29):  # This response the available class, A-Z, nothing, space, and delete
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.develop_model()

    def develop_model(self):
        # determine the input layer shape
        model = tf.keras.Sequential([
            layers.Input(shape=self.input_shape),  # shrink the images
            layers.Conv2D(32, 3, activation='relu'),

            layers.MaxPooling2D(),  # reduce more of the dimensions
            layers.Conv2D(64, 3, activation='relu'),

            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, activation='relu'),

            layers.MaxPooling2D(),
            layers.Flatten(),

            layers.Dense(64, activation='relu'),
            layers.Dense(self.num_classes, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model


def main():
    # Dataset path
    DATASET_PATH = r'E:\Documents\King Of The Software Engineers\ML\DATA 5100 Programming for Data Science\Final Porject\archive\asl_alphabet_train\asl_alphabet_train'
    MODEL_SAVE_PATH = os.path.join(os.path.dirname(DATASET_PATH), 'best_sign_language_model.keras')

    # Data augmentation and preprocessing
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        validation_split=0.2
    )

    # Display Class options
    print(f"The options classes: {sorted(os.listdir(DATASET_PATH))}")

    # Prepare data
    train_generator = train_datagen.flow_from_directory(
        DATASET_PATH,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        DATASET_PATH,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        shuffle=True
    )

    # Initialize the model
    sign_model = SignModel(num_classes=len(os.listdir(DATASET_PATH)))

    checkpoint = ModelCheckpoint(
        MODEL_SAVE_PATH,
        monitor='val_accuracy',
        save_best_only=True
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # traning the model
    history = sign_model.model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=50,
        callbacks=[checkpoint, early_stopping]
    )

    # display evaluation
    final_accuracy = history.history['accuracy'][-1]
    final_val_accuracy = history.history['val_accuracy'][-1]
    print(f"\nModel training is now complete!")
    print(f"The training accuracy: {final_accuracy:.4f}")
    print(f"The validation accuracy: {final_val_accuracy:.4f}")


if __name__ == '__main__':
    main()


In [ ]:
# this is the GUI.py

import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import os
from tkinter import PhotoImage


class SignLanguageInterpreter:
    def __init__(self):

        # # Initialize the Tkinter
        self.root = tk.Tk()

        # Set the GUI size, width and height
        self.root.geometry('1000x800')
        # Set the GUI title
        self.root.title("Sign Language Interpreter")
        # path for sign language images
        # It is stored in my local machine
        # We can also use Kaggle API however I find this more convenient
        self.images_path = (r'E:\Documents\King Of The Software Engineers\ML\DATA 5100 Programming for Data '
                            r'Science\Final Porject\archive\asl_alphabet_train\asl_alphabet_train')

        self.make_widgets()
        # Change the background color
        self.root.configure(bg="lightgreen")
        # Tinker logo
        # Credit  due to https://www.flaticon.com/search?word=Sign%20Language
        self.root.iconphoto(True, PhotoImage(file="sign-language.png"))

    def make_widgets(self):

        self.text_input = tk.Entry(self.root, width=50, font=('Comic Sans MS', 14))
        self.input_label = tk.Label(self.root, text="Translate Text:", font=('Comic Sans MS', 14))

        self.text_input.pack(pady=11)
        self.input_label.pack(pady=11)

        # Translate button, assigning job to the buttons
        self.btn_translate = (tk.Button
            (
            self.root,
            text="Translate",
            command=self.translation,
            font=('Comic Sans MS', 14)
        ))
        self.btn_translate.pack(pady=11)

        self.image_frame = tk.Frame(self.root)
        self.image_frame.pack(pady=22, expand=True, fill='both')

    def translation(self):
        # Clear the cash images
        for widget in self.image_frame.winfo_children():
            widget.destroy()

        text = self.text_input.get().upper()
        row = tk.Frame(self.image_frame)
        row.pack()

        for i, c in enumerate(text):
            if c == c.isspace():

                image_path = os.path.join(self.images_path, 'space')
            elif c.isalpha():

                image_path = os.path.join(self.images_path, c)
            else:
                continue

            # Find the first images
            try:
                image_file = os.listdir(image_path)[0]
                image_path = os.path.join(image_path, image_file)

                # Load the images
                img = Image.open(image_path)
                # adjust for display
                img = img.resize((95, 95))
                img = ImageTk.PhotoImage(img)

                # Make the label then display the images
                lbl = tk.Label(row, image=img)
                lbl.image = img
                lbl.pack(side=tk.LEFT, padx=5)

                # make a new row for every 5 characters
                if (i + 1) % 5 == 0:
                    row = tk.Frame(self.image_frame)
                    row.pack()

            except Exception as e:
                print(f"Failed to load image for the {c}: {e}")

    def start(self):
        self.root.mainloop()


if __name__ == "__main__":
    app = SignLanguageInterpreter()
    app.start()
